In [4]:
import os

In [2]:
def get_proj_root(proj_root_marker_file='proj_root.here'):
    
    max_depth = 10
    
    def __get_proj_root(folder_path, depth):
        if depth == 0:
            print('Project root (project_root.here file) is not found!')
            return None
        
        onlyfiles = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        if proj_root_marker_file in onlyfiles:
            return folder_path
        else:
            parent_dir = os.path.abspath(os.path.join(folder_path, os.pardir))
            return __get_proj_root(parent_dir, depth-1)
    
    return __get_proj_root(os.getcwd(), max_depth)

In [5]:
# As we are in a Jupyter Notebook - we have to change current working directory 
os.chdir('../../')
os.getcwd()    

'/mnt'

In [15]:
import sys
import logging
import time
from argparse import ArgumentParser

import torch
import torch.nn as nn

sys.path.append('/mnt')
sys.path.append('/mnt/skin_cancer_nas')
sys.path.append('/mnt/skin_cancer_nas/data/torch_generator')
from skin_cancer_nas.data.torch_generator import generator as data_gen  
from skin_cancer_nas.data.torch_generator import base_classes
from skin_cancer_nas.data.torch_generator import config
from skin_cancer_nas.data.torch_generator import preprocessor

# import datasets
import nas.darts_torch.datasets
# from model_melanoma import Combine, CNN
from nas.darts_torch.model_melanoma import Combine, CNN
from nni.nas.pytorch.callbacks import ArchitectureCheckpoint, LRSchedulerCallback
from nni.nas.pytorch.darts import DartsTrainer
# from utils import accuracy
from nas.darts_torch.utils import accuracy
# from base_classes import Dataset
# from nas.darts_torch.base_classes import Dataset

from torchvision import transforms

logger = logging.getLogger('nni')

if __name__ == "__main__":
    parser = ArgumentParser("darts")
    parser.add_argument("--layers", default=8, type=int)
    parser.add_argument("--batch-size", default=4, type=int)
    parser.add_argument("--log-frequency", default=10, type=int)
    parser.add_argument("--epochs", default=50, type=int)
    parser.add_argument("--channels", default=16, type=int)
    parser.add_argument("--unrolled", default=False, action="store_true")
    parser.add_argument("--visualization", default=False, action="store_true")
    args = parser.parse_args()

    # dataset_train, dataset_valid = datasets.get_dataset("cifar10")
    partition, labels = data_gen.train_val_split(val_size=0.1)

    MEAN = [0.2336, 0.6011, 0.3576, 0.4543]
    STD = [0.0530, 0.0998, 0.0965, 0.1170]
    normalize = [
        transforms.Normalize(MEAN, STD)
    ]
    train_transform = transforms.Compose(normalize)
    valid_transform = transforms.Compose(normalize)
    # Generators Declaration
    training_set = Dataset(partition['train'], labels, transform=train_transform)
    training_generator = torch.utils.data.DataLoader(training_set, **data_gen.PARAMS)
    validation_set = Dataset(partition['validation'], labels, transform=valid_transform)
    validation_generator = torch.utils.data.DataLoader(validation_set, **data_gen.PARAMS)
                    
    cnn_model = CNN(input_size=256, 
                    in_channels=4, 
                    channels=args.channels, 
                    n_classes=len(set(labels.values())), 
                    n_layers=args.layers)
    # combined_model = Combine(cnn_model)
    criterion = nn.CrossEntropyLoss()

    optim = torch.optim.SGD(cnn_model.parameters(recurse=True), 0.025, momentum=0.9, weight_decay=3.0E-4)
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, args.epochs, eta_min=0.001)

    trainer = DartsTrainer(cnn_model,
                           loss=criterion,
                           metrics=lambda output, target: accuracy(output, target, topk=(1,)),
                           optimizer=optim,
                           num_epochs=args.epochs,
                           dataset_train=training_set,
                           dataset_valid=validation_set,
                           batch_size=args.batch_size,
                           log_frequency=args.log_frequency,
                           unrolled=args.unrolled,
                           workers=0,
                           callbacks=[LRSchedulerCallback(lr_scheduler), ArchitectureCheckpoint("./checkpoints")])
    
    if args.visualization:
        trainer.enable_visualization()
    trainer.train()

usage: darts [-h] [--layers LAYERS] [--batch-size BATCH_SIZE]
             [--log-frequency LOG_FREQUENCY] [--epochs EPOCHS]
             [--channels CHANNELS] [--unrolled] [--visualization]
darts: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-dd4d3f2e-5757-4357-8836-7b3386a5d07c.json


SystemExit: 2

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
!pip list

Package                       Version
----------------------------- -----------
absl-py                       0.9.0
alabaster                     0.7.12
astor                         0.8.1
astunparse                    1.6.3
attrs                         19.3.0
autokeras                     1.0.3
awscli                        1.18.86
Babel                         2.8.0
backcall                      0.2.0
bleach                        3.1.5
botocore                      1.17.9
cachetools                    4.1.0
catboost                      0.23.2
certifi                       2020.6.20
cffi                          1.14.0
chardet                       3.0.4
click                         7.1.2
cmake                         3.17.3
colorama                      0.4.3
contextlib2                   0.6.0.post1
coverage                      5.1
cycler                        0.10.0
decorator                     4.4.2
defusedxml                    0.6.0
docopt                        0.6.2
doc

In [11]:
!pip install tensorflow-gpu --upgrade

     |################################| 516.2 MB 58 kB/s s eta 0:00:01     |#################               | 279.0 MB 13.2 MB/s eta 0:00:18     |#####################           | 346.5 MB 29.3 MB/s eta 0:00:06
  Using cached tensorboard-2.2.2-py3-none-any.whl (3.0 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.13.0
    Uninstalling tensorboard-1.13.0:
      Successfully uninstalled tensorboard-1.13.0
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 1.10.0
    Uninstalling tensorflow-gpu-1.10.0:
      Successfully uninstalled tensorflow-gpu-1.10.0


In [12]:
!pip install keras --upgrade

  Attempting uninstall: keras
    Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6


In [16]:
!ls

Dockerfile		    SimpleCNN_triple-subreg_v6	notebooks
LICENSE			    checkpoints			project_root.here
Makefile		    data			references
README.md		    docker-compose.yml		reports
SimpleCNN_triple-subreg_v1  docker_clean_all.sh		requirements copy.txt
SimpleCNN_triple-subreg_v2  docs			requirements.txt
SimpleCNN_triple-subreg_v3  image_classifier		skin_cancer_nas
SimpleCNN_triple-subreg_v4  logs			start.sh
SimpleCNN_triple-subreg_v5  models			tox.ini
